In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Flatten,Reshape
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
import csv
import nltk
from tensorflow.keras.models import Model

In [2]:
from tensorflow.keras import backend as k
from tensorflow.keras.layers import Input,Concatenate,Dropout,Dense,BatchNormalization,Conv1D
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
import scipy
from tensorflow.keras.initializers import he_normal,glorot_normal
from tensorflow.keras.regularizers import l1,l2
from tensorflow.python.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint,LearningRateScheduler,ReduceLROnPlateau
from time import time
from tensorflow.keras.utils import plot_model
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input,BatchNormalization,Dropout
from tensorflow.keras import optimizers
import random as rn
import string
from sklearn.metrics import f1_score
from tensorflow import keras
import numpy as np
import datetime
import os
import math
import pandas as pd
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.text import one_hot
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.initializers import glorot_uniform,glorot_normal
from tensorflow.keras.layers import MaxPooling1D

In [96]:
#loading the training data
import pandas as pd
training_data=pd.read_csv("./train.csv")
display(train.head(2))
#display(test.head(2))

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,compound,negative,positive,neutral,com_len
0,0016926B079C,i think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0,0.9363,0.050,0.098,0.852,264
1,0022683E9EA5,when a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5,-0.9194,0.111,0.091,0.798,536


In [97]:

 """
     ref: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python/47091490#47091490"""
def decontractions(phrase):
   
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"won\’t", "will not", phrase)
    phrase = re.sub(r"can\’t", "can not", phrase)
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"n\’t", " not", phrase)
    phrase = re.sub(r"\’re", " are", phrase)
    phrase = re.sub(r"\’s", " is", phrase)
    phrase = re.sub(r"\’d", " would", phrase)
    phrase = re.sub(r"\’ll", " will", phrase)
    phrase = re.sub(r"\’t", " not", phrase)
    phrase = re.sub(r"\’ve", " have", phrase)
    phrase = re.sub(r"\’m", " am", phrase)

    return phrase
#preprocessing 
from nltk.corpus import stopwords
from tqdm import tqdm
import re
stopwords = stopwords.words('english')
def preprocess(text_col,stopword):
    preprocessed = []
    for sentence in tqdm(text_col.values):
        # Replace "carriage return" with "space".
        sentence=str(sentence)
        sent = sentence.replace('\\r', ' ')
        # Replace "quotes" with "space".
        sent = sent.replace('\\"', ' ')
        # Replace "line feed" with "space".
        sent = sent.replace('\\n', ' ')
         # Replace characters between words with "space".
        sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
        #remove stop words
        #decontraction
        sent=decontractions(sent)
        if stopword:
            sent = ' '.join(e for e in sent.split() if e not in stopwords)
        else:
            sent = ' '.join(e for e in sent.split())
        # to lowercase
        preprocessed.append(sent.lower().strip())
    return preprocessed
training_data['full_text']=preprocess(training_data['full_text'],stopword=False)
#test['full_text']=preprocess(test['full_text'],stopword=False)

100%|█████████████████████████████████████████████████████| 3911/3911 [00:03<00:00, 1045.96it/s]


In [98]:
training_data.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,i think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,when a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,dear principal if u change the school policy o...,3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,the best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [10]:
#cehcking word count 
train['length']=train['full_text'].apply(lambda x:len(x.split()))
test['length']=test['full_text'].apply(lambda x:len(x.split()))
train.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,compound,negative,positive,neutral,com_len
0,0016926B079C,i think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0,0.9363,0.050,0.098,0.852,264
1,0022683E9EA5,when a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5,-0.9194,0.111,0.091,0.798,536
2,00299B378633,dear principal if u change the school policy o...,3.0,3.5,3.0,3.0,3.0,2.5,0.9271,0.054,0.110,0.836,330
3,003885A45F42,the best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0,0.9996,0.051,0.252,0.698,756
4,0049B1DF5CCC,small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5,0.9868,0.000,0.164,0.836,234


In [99]:
y=training_data[['cohesion','syntax','vocabulary','phraseology','grammar','conventions']]
X=training_data.drop(['text_id','cohesion','syntax','vocabulary','phraseology','grammar','conventions'],axis=1)
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.20)
print(X_train.shape,y_train.shape)



(3128, 1) (3128, 6)


In [100]:
#padding 

def pad_text(text,tokenizer,max_len):
    return pad_sequences(tokenizer.texts_to_sequences(text),maxlen=max_len,padding='post')


def text_padding(train,test,test_final,max_len):
    vocab=5000
    token=Tokenizer()
    token.fit_on_texts(train)
    padded_train_text=pad_text(train,token,max_len)
    padded_test_text=pad_text(test,token,max_len)
    padded_test_final_text=pad_text(test_final,token,max_len)
    return padded_train_text,padded_test_text,padded_test_final_text,token
comm_len=200
train_com_pad,test_com_pad,test_final_pad,token_com= text_padding(X_train['full_text'],X_test['full_text'],test['full_text'],comm_len)
print(train_com_pad.shape,test_com_pad.shape,test_final_pad.shape)

(3128, 200) (783, 200) (3, 200)


In [101]:
print(train_com_pad)

[[ 446    3 3950 ...  274   12   23]
 [  12 2510   72 ... 5965   12 1188]
 [  77   58   19 ...   15  105   11]
 ...
 [  12    2  327 ...  116  619  390]
 [ 569   83  454 ...    5  636   85]
 [  47   66   26 ...   36   11  238]]


In [102]:
def generate_embedding_matrix(token):
    embedding_path='./crawl-300d-2M.vec' #pre trained FastText English word vectors released by FB
    embedding_size=300
    vocab_size=5000
    embedding_index={}
    with open(embedding_path, 'r',encoding="utf8") as f:
         for line in f:
                values = line.rstrip().rsplit(' ')
                word = values[0]
                coefs = np.asarray(values[1:], dtype='float32')
                embedding_index[word] = coefs
    num_words = len(token.word_index) + 1
    embedding_matrix = np.zeros((num_words, embedding_size))
    for word, i in token.word_index.items():
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [103]:
# generating the embedding matrix containing weigths
embedding_comm = generate_embedding_matrix(token_com)
print(embedding_comm.shape)

(18974, 300)


In [104]:
print(train_s.shape,test_s.shape,test_final.shape)

(3128, 6) (783, 6) (3, 5)


In [105]:
train_s=np.array(y_train)
test_s=np.array(y_test)

In [106]:
X_train=[train_com_pad,train_com_pad,train_s]
X_test=[test_com_pad,test_com_pad,test_s]
#X_test_final=[test_final_pad,test_final_pad,test_final]
y_train=np.array(y_train,dtype=np.float64)
y_test=np.array(y_test, dtype=np.float64)
def mcrmse(y_true, y_pred):
    colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=0)
    return tf.reduce_mean(tf.sqrt(colwise_mse), axis=0)

In [107]:
adam = tf.keras.optimizers.Adam(learning_rate=0.001)
def LSTM_CNN1D(comm_len,token_com):
    drop_lstm = 0.25
    drop_dense = 0.25
    num_lstm=150
    input_1 = Input(shape=(comm_len,),name = 'input_comment_1')
    embedding_layer_1 = Embedding(len(token_com.word_index) + 1,300,weights=[embedding_comm],input_length=comm_len,trainable=False,dtype=tf.float32)(input_1)
    conv_1_1 = Conv1D(64,3,strides=1, padding='same',activation='relu')(embedding_layer_1)
    lstm_1_1 = LSTM(64,dropout=drop_lstm,return_sequences=True,dtype=tf.float32)(embedding_layer_1)
    concate_1 = keras.layers.Concatenate(axis=-1)([conv_1_1, lstm_1_1])
    flatten_1 = Flatten()(concate_1)

    # creating layers for parent comment data
    input_2 = Input(shape=(comm_len,),name = 'input_comment_2')
    embedding_layer_2 = Embedding(len(token_com.word_index) + 1,300,weights=[embedding_comm],input_length=comm_len,trainable=False,dtype=tf.float32)(input_2)
    conv_1_1 = Conv1D(128,3,strides=1, padding='same',activation='relu')(embedding_layer_2)
    lstm_1_2 =LSTM(128,dropout=drop_lstm,return_sequences=True,dtype=tf.float32)(embedding_layer_2)
    concate_2 = keras.layers.Concatenate(axis=-1)([conv_1_1, lstm_1_2])
    flatten_2 = Flatten()(concate_2)

    # creating layers for numerical columns
    input_3 = Input(shape=(6,), name = "input_numerical")
    dense_num_layer = Dense(128, activation = "relu")(input_3)

    concatenated_layer = keras.layers.concatenate([flatten_1,flatten_2,dense_num_layer],axis=-1)

    # creating further layers
    x = Dense(128, activation = 'relu',kernel_initializer=glorot_uniform(seed=42))(concatenated_layer)
    x = BatchNormalization()(x)
    x = Dense(64, activation = 'relu',kernel_initializer=glorot_uniform(seed=42))(x)
    x = BatchNormalization()(x)
    x = Dense(32, activation = 'relu',kernel_initializer=glorot_uniform(seed=42))(x)
    x = BatchNormalization()(x)
    x = Dense(16, activation = 'relu',kernel_initializer=glorot_uniform(seed=42))(x)
    output = Dense(6,activation='linear')(x)
    model = Model(inputs = [input_1,input_2,input_3], outputs = [output])
    model.compile(optimizer=adam, loss = mcrmse, metrics = mcrmse)
    return model

In [108]:
model=LSTM_CNN1D(comm_len,token_com)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_comment_1 (InputLayer)   [(None, 200)]        0           []                               
                                                                                                  
 input_comment_2 (InputLayer)   [(None, 200)]        0           []                               
                                                                                                  
 embedding_6 (Embedding)        (None, 200, 300)     5692200     ['input_comment_1[0][0]']        
                                                                                                  
 embedding_7 (Embedding)        (None, 200, 300)     5692200     ['input_comment_2[0][0]']        
                                                                                            

In [66]:
#reduce_lr reduces the learning rate when the metric has stoppes improving for 2 epochs. 
#Using EarlyStopping to stop the calculation upon reaching enough accuracy
reduce_lr = ReduceLROnPlateau(monitor = 'val_mcrmse', factor = 0.25, patience = 2, verbose = 1)
earlystop = EarlyStopping(monitor = 'val_mcrmse',  mode="min",min_delta = 0, patience = 25,verbose = 1)
callbacks = [reduce_lr,earlystop]

In [109]:
hitory1=model.fit(x=X_train,y=y_train,epochs=300,batch_size=32,validation_data=(X_test, y_test),callbacks=callbacks)

Epoch 1/300
98/98 [==============================] - 82s 620ms/step - loss: 1.5571 - mcrmse: 1.5551 - val_loss: 1.8791 - val_mcrmse: 1.8823 - lr: 0.0010
Epoch 2/300
98/98 [==============================] - 44s 450ms/step - loss: 0.4874 - mcrmse: 0.4874 - val_loss: 1.1480 - val_mcrmse: 1.1484 - lr: 0.0010
Epoch 3/300
98/98 [==============================] - 52s 527ms/step - loss: 0.4295 - mcrmse: 0.4293 - val_loss: 0.5263 - val_mcrmse: 0.5270 - lr: 0.0010
Epoch 4/300
98/98 [==============================] - 60s 611ms/step - loss: 0.3865 - mcrmse: 0.3866 - val_loss: 0.4638 - val_mcrmse: 0.4640 - lr: 0.0010
Epoch 5/300
98/98 [==============================] - 55s 564ms/step - loss: 0.3438 - mcrmse: 0.3439 - val_loss: 0.4215 - val_mcrmse: 0.4212 - lr: 0.0010
Epoch 6/300
98/98 [==============================] - 58s 593ms/step - loss: 0.3310 - mcrmse: 0.3312 - val_loss: 0.4853 - val_mcrmse: 0.4859 - lr: 0.0010
Epoch 7/300
98/98 [==============================] - 55s 557ms/step - loss: 0.3153

Epoch 47/300
98/98 [==============================] - 61s 624ms/step - loss: 0.2155 - mcrmse: 0.2159 - val_loss: 0.1401 - val_mcrmse: 0.1404 - lr: 6.1035e-08
Epoch 48/300
98/98 [==============================] - ETA: 0s - loss: 0.2161 - mcrmse: 0.2163
Epoch 48: ReduceLROnPlateau reducing learning rate to 1.525878978725359e-08.
98/98 [==============================] - 59s 601ms/step - loss: 0.2161 - mcrmse: 0.2163 - val_loss: 0.1405 - val_mcrmse: 0.1408 - lr: 6.1035e-08
Epoch 49/300
98/98 [==============================] - 57s 586ms/step - loss: 0.2076 - mcrmse: 0.2080 - val_loss: 0.1401 - val_mcrmse: 0.1404 - lr: 1.5259e-08
Epoch 50/300
98/98 [==============================] - ETA: 0s - loss: 0.2174 - mcrmse: 0.2175
Epoch 50: ReduceLROnPlateau reducing learning rate to 3.814697446813398e-09.
98/98 [==============================] - 56s 576ms/step - loss: 0.2174 - mcrmse: 0.2175 - val_loss: 0.1407 - val_mcrmse: 0.1409 - lr: 1.5259e-08
Epoch 51/300
98/98 [==============================] 

Epoch 82/300
98/98 [==============================] - ETA: 0s - loss: 0.2241 - mcrmse: 0.2240
Epoch 82: ReduceLROnPlateau reducing learning rate to 8.881784618863365e-19.
98/98 [==============================] - 55s 562ms/step - loss: 0.2241 - mcrmse: 0.2240 - val_loss: 0.1400 - val_mcrmse: 0.1403 - lr: 3.5527e-18
Epoch 83/300
98/98 [==============================] - 61s 623ms/step - loss: 0.2167 - mcrmse: 0.2169 - val_loss: 0.1395 - val_mcrmse: 0.1398 - lr: 8.8818e-19
Epoch 84/300
98/98 [==============================] - 56s 576ms/step - loss: 0.2150 - mcrmse: 0.2152 - val_loss: 0.1398 - val_mcrmse: 0.1401 - lr: 8.8818e-19
Epoch 85/300
98/98 [==============================] - ETA: 0s - loss: 0.2103 - mcrmse: 0.2106
Epoch 85: ReduceLROnPlateau reducing learning rate to 2.220446154715841e-19.
98/98 [==============================] - 57s 578ms/step - loss: 0.2103 - mcrmse: 0.2106 - val_loss: 0.1402 - val_mcrmse: 0.1404 - lr: 8.8818e-19
Epoch 86/300
98/98 [==============================] 

In [110]:
y_pred=model.predict(X_test)

25/25 [==============================] - 4s 107ms/step


In [111]:
print(y_pred)

[[2.060544  2.8073852 3.1038997 2.9284852 2.7050638 2.3144805]
 [4.2346964 3.9889119 3.8833306 4.403566  3.6411748 3.9725156]
 [3.2831786 3.152833  3.278423  3.1306183 3.1861482 3.7059994]
 ...
 [3.336904  2.406166  3.1063833 2.6997616 2.6479027 3.262218 ]
 [3.4932804 3.4040241 3.9232392 3.5167248 2.963857  3.5503461]
 [2.5805817 2.5822735 2.6630604 2.996856  2.1830955 2.1393182]]


In [112]:
from sklearn.metrics import mean_squared_error
for i in range(6):
    error = mean_squared_error(y_test[:,i],y_pred[:,i])
    print(error)

0.016994304263453806
0.02690464616531071
0.023637707888579624
0.016834125368980116
0.02222512603538461
0.015453030787441977


In [ ]:
sample=pd.read_csv("/content/drive/MyDrive/LSTM_network/sample_submission.csv")

In [113]:
Test_data = pd.read_csv("./test.csv")

In [117]:
test_final=np.ones((3,6),dtype=float)
X_test_final=[test_final_pad,test_final_pad,test_final]

In [120]:
from tensorflow.keras.models import Sequential, model_from_json
model_json = model.to_json()
with open("model_trial.json","w") as json_file:
    json_file.write(model_json)

In [121]:
model.save_weights("weights_trial.h5")

In [88]:
# loading the model and weights
json_file=open('model_trial.json','r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("weights_trial.h5")

In [124]:
X_train_rand = [train_com_pad,train_com_pad,train_s];
y_pred=model.predict(X_train_rand)
print(y_pred.shape)

98/98 [==============================] - 12s 124ms/step
(3128, 6)


In [125]:
for i in range(6):
    error = mean_squared_error(train_s[:,i],y_pred[:,i])
    print(error)

0.011125922495189481
0.015309660592061305
0.01340009335178853
0.010513719649870733
0.014354437100536134
0.0085495108602342


In [118]:
y_pred=model.predict(X_test_final)

1/1 [==============================] - 0s 91ms/step


In [119]:
print(y_pred)

[[1.2007238  1.1055958  1.5844516  1.6045799  1.1289417  1.7898675 ]
 [0.76105666 0.70036066 0.9543434  0.92761356 0.6785991  1.0206932 ]
 [1.0990517  1.1511546  1.6719697  1.534158   1.068452   1.7473599 ]]


In [ ]:
sample.to_csv('submission.csv',index=False)